# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [1]:
from utils import get_dataset

In [3]:
get_dataset?

In [4]:
dataset = get_dataset("/home/michal/waymo/processed/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord")

INFO:tensorflow:Reading unweighted datasets: ['/home/michal/waymo/processed/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord']
INFO:tensorflow:Reading record datasets for input file: ['/home/michal/waymo/processed/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord']
INFO:tensorflow:Number of filenames to read: 1
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.deterministic`.
Instructions for updating:
Use `tf.data.Dataset.map()


## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [ ]:
def display_instances(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes.
    """
    # ADD CODE HERE

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [ ]:
## STUDENT SOLUTION HERE

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...

In [5]:
for r in dataset.take(1):
    pass

In [8]:
r.keys()

dict_keys(['image', 'source_id', 'key', 'filename', 'groundtruth_image_confidences', 'groundtruth_verified_neg_classes', 'groundtruth_not_exhaustive_classes', 'groundtruth_boxes', 'groundtruth_area', 'groundtruth_is_crowd', 'groundtruth_difficult', 'groundtruth_group_of', 'groundtruth_weights', 'groundtruth_classes', 'groundtruth_image_classes', 'original_image_spatial_shape'])

In [9]:
r["image"]

<tf.Tensor: shape=(640, 640, 3), dtype=uint8, numpy=
array([[[161, 204, 255],
        [149, 192, 243],
        [155, 198, 249],
        ...,
        [ 47,  66,  73],
        [ 49,  68,  75],
        [ 50,  69,  76]],

       [[157, 200, 251],
        [152, 195, 246],
        [155, 198, 249],
        ...,
        [ 49,  68,  75],
        [ 48,  67,  74],
        [ 47,  66,  73]],

       [[157, 203, 252],
        [150, 196, 245],
        [157, 203, 253],
        ...,
        [ 51,  70,  76],
        [ 49,  68,  74],
        [ 50,  69,  75]],

       ...,

       [[ 53,  70,  80],
        [ 53,  71,  81],
        [ 54,  74,  83],
        ...,
        [ 27,  49,  62],
        [ 27,  51,  61],
        [ 24,  48,  58]],

       [[ 53,  70,  80],
        [ 54,  72,  82],
        [ 52,  72,  81],
        ...,
        [ 25,  47,  60],
        [ 24,  48,  60],
        [ 22,  46,  58]],

       [[ 52,  69,  79],
        [ 54,  72,  82],
        [ 49,  69,  78],
        ...,
        [ 25,  47,  6